In [3]:
!ls

Distribution.py RV.py           __pycache__
Funcs.py        Sample.py       scratch.ipynb


In [62]:
import numpy as np
import math
from RV import RV
from Sample import Sample
import scipy.stats as sci

class Distribution:
    
    def plot_pdf(self):
        pass

    def plot_cdf(self):
        pass

    def draw(self, n=1):
        if n==1:
            return RV(distr=self)
        else:
            return Sample([RV(distr=self) for i in range(n)])


class Normal(Distribution):
    def __init__(self, mean:float, var:float):
        self.mean = mean
        self.var = var
        self.std = np.sqrt(self.var) #Add Handling negative variance
        self.dist = sci.norm(loc=self.mean, scale=self.std)
        self.support = (-np.inf, np.inf)

    def __str__(self):
        return f"N(mean={self.mean}, variance={self.var})"
    
    def pdf(self, x):
        return self.dist.pdf(x)
    
    def cdf(self, x):
        return self.dist.cdf(x)
    
    def mgf(self, t):
        return self.dist.mgf(t)


class Uniform(Distribution):
    def __init__(self, a, b):
        self.a, self.b = a, b
        self.E = (self.a+self.b)/2
        self.dist = sci.uniform(loc=a, scale=b-a)
    
    def pdf(self, x):
        return self.dist.pdf(x)
    
    def cdf(self, x):
        return self.dist.cdf(x)
    
    def mgf(self, x):
        return self.dist.mgf(x)


class Binomial(Distribution):
    def __init__(self, n:int, p:float):
        self.n = n
        self.p = p
        self.dist = sci.binom(n=self.n, p=self.p)

    def pmf(self, x):
        return self.dist.pmf(x)
    
    def cdf(self, x):
        return self.dist.cdf(x)


class Poisson(Distribution):
    def __init__(self, mu):
        self.mu = mu
        self.dist = sci.poisson(mu)

    def pmf(self, x):
        return self.dist.pmf(x)
    
    def cdf(self, x):
        return self.dist.cdf(x)


class Exponential(Distribution):
    def __init__(self, intensity: float):
        self.intensity = intensity
        self.dist = sci.expon(scale=1/intensity)

    def pdf(self, x):
        return self.dist.pdf(x)
    
    def cdf(self, x):
        return self.dist.cdf(x)

    def mgf(self, t):
        return self.intensity / (self.intensity - t)


# class Gamma(Distribution):
#     def __init__(self, theta, r):
#         self.theta = theta
#         self.r = r
#         self.dist = sci.gamma()

#     def pdf(self, x):
#         return self.dist.pdf(x)
    
#     def cdf(self, x):
#         return self.dist.cdf(x)

Distr = Distribution
N = Normal

if __name__ == "__main__":
    X = RV(distr=Binomial(n=20, p=0.1))
    print(X.distribution)


In [63]:
class RV:
    def __init__(self, distr:"Distribution", name=None):
        self.distribution = distr
        self.name = name

    def realize(self):
        return self.distribution.dist.rvs()
    
    def __str__(self):
        return self.name

In [64]:
class Sample:
    def __init__(self, rvs:list["RV"]):
        self.rvs = rvs

    def observe(self):
        return np.array([rv.realize() for rv in self.rvs])
    
    def __str__(self):
        return []


In [65]:
sample = Normal(0, 1).draw(10)

## Central Limit Theorem

In [79]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=2)

distributions = [Normal(0, 1), Exponential(1), Poisson(1), Binomial(10, 0.4)]

for i, distr in enumerate(distributions):
    abstract_sample = distr.draw(n=2)
    sample_means = [np.mean(abstract_sample.observe()) for i in range(200)]
    fig.add_trace(
        go.Histogram(x=sample_means),
        row=i//2 + 1, col=i%2 + 1
    )

fig.update_layout(height=500, width=650, title_text="Side By Side Subplots")
fig.show()
